# Detecting Epileptic Seizures through EEG Data: Part 3

So far, we have used KNNs, logistic regression, simple NNs and CNNs for epileptic seizure detection from EEG. Since EEG is a time-series, let's explore how to use time-series-specific model, recurrent neural networks, to detect seizures!

## Goals for today:
1. Seizure detection using a simple RNN
2. Seizure detection using a Long Short Term Memory (LSTM) network
3. Many-to-many sequence modeling
4. Visualizing model confidence over time

In [ ]:
#@title ##Import libraries!

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn import model_selection
from sklearn.model_selection import train_test_split

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, SimpleRNN, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
import keras.optimizers as optimizers
from keras.callbacks import ModelCheckpoint
monitor = ModelCheckpoint('./model.hdf5', 
                          monitor='val_accuracy', 
                          verbose=0, 
                          save_best_only=True, 
                          save_weights_only=False, 
                          mode='auto', 
                          save_freq='epoch')

import gdown

## Set random seed for reproducible results
RAND_SEED = 12
np.random.seed(RAND_SEED)
tf.random.set_seed(RAND_SEED)

## Utils function to combine 23 chunks from the same patient into one big chunk
def prepare_data(eeg_df):
  file_names = eeg_df['Unnamed: 0'].tolist()

  subject_ids = []
  chunk_ids = []
  for fn in file_names:
    subject_ids.append(fn.split('.')[-1])
    chunk_ids.append(fn.split('.')[0])
  subject_ids = list(set(subject_ids))
  assert len(subject_ids) == 500

  sub2ind = {}
  for ind, sub in enumerate(subject_ids):
    sub2ind[sub] = ind

  eeg_combined = np.zeros((500, int(178*23)))
  labels_combined = np.zeros(500)
  labels_chunks = np.zeros((500, 23))
  labels_dict = {}
  for i in range(len(eeg_df)):
    fn = eeg_df.iloc[i]['Unnamed: 0']
    subject_id = fn.split('.')[-1]
    subject_ind = sub2ind[subject_id]

    chunk_id = int(fn.split('.')[0].split('X')[-1])
    start_idx = (chunk_id - 1) * 178
    end_idx = start_idx + 178
    eeg_combined[subject_ind, start_idx:end_idx] = eeg_df.iloc[i].values[1:-1]

    if subject_id not in labels_dict:
      labels_dict[subject_id] = []
    labels_dict[subject_id].append(eeg_df.iloc[i].values[-1])

  for sub_id, labels in labels_dict.items():
    sub_ind = sub2ind[sub_id]
    is_seizure = int(np.any(np.array(labels) == 1))
    labels_combined[sub_ind] = is_seizure
    labels = np.array(labels)
    labels = np.where(labels>1, 0, labels)
    labels_chunks[sub_ind,:] = labels

  return eeg_combined, labels_combined, labels_chunks


In [ ]:
#@title ## Download our data set!
data_path = 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/AI%20%2B%20Healthcare/Projects%20(Session%206%2B)/Seizure%20Prediction%20/data.csv'
uci_epilepsy = './uci_epilepsy'
gdown.download(data_path, uci_epilepsy, False)

# Let's revisit the EEG data

In [ ]:
eeg = pd.read_csv(uci_epilepsy)
eeg.head()

## Visualizing one EEG sequence

In [ ]:
eeg_all, _, _ = prepare_data(eeg)

eeg1 = np.array(eeg_all[0,:]) # take the first EEG
print('Length of one EEG sequence: ', len(eeg1))
print(eeg1)

In [ ]:
x = np.linspace(0, 23, 4094)

plt.plot(x,eeg1)
plt.show()

#### **Question**: What are the x-axis and y-axis here?

## Prepare input data for RNNs

RNNs are designed for **sequence data**: the model processes its input and produces output **in order**. 

Check out [these slides](https://docs.google.com/presentation/d/1ykLNZNkql0SDqqDiNLKJVspUFhVLKB_7x-uB7sSlbNI/) for more.

The "vanilla" version of RNNs works as follows. As a vanilla RNN processes the input, it keeps a **hidden state vector** to remember some of what it's seen so far. At each step with each **new input**, the model uses the **old hidden state** to compute a **new hidden state** and a **new output**.

For our seizure detection task, the input is an EEG sequence, and the output is one single label indicating seizure or no seizure. Therefore, we can use the **final hidden state** as the representation for the whole sequence, and feed it to fully connected layer(s) for seizure detection! This is often called **many-to-one** sequence modeling.

In our dataset, each sample consists of 23 seconds of EEG recordings, and each second has 178 sampled data points. Therefore, we can treat one second as one **time step**, and thus each time step is a vector of length 178.

[//]: <> (Here, we will treat each of the 178 data points as one **time step**, and thus each time step is simply a continuous number, i.e. the feature dimension is 1.)

An RNN requires its input to have shape `(number of time steps, feature dimension)`. Let's convert `eeg1` into the required shape. 

In [ ]:
# Convert eeg1 to the input shape required by an RNN
print('eeg1 shape before reshape: ', eeg1.shape)
### YOUR CODE HERE

### END CODE
print('eeg1 shape after reshape: ', eeg1.shape) 

# Build and Evaluate a vanilla RNN

### First, extract input and output data

Our input data `x` are EEG sequences, and output data `y` are binary labels indicating seizure or no seizure.

In [ ]:
# Get x (EEG data) and y (binary classes --- 1 for seizure and 0 for no seizure)
# y_time_steps here has one label for each time step, will be used later
x, y, y_time_steps = prepare_data(eeg)
x = x.reshape(-1, 23, 178).astype(np.float32) # reshape x into (number_of_samples, number_of_time_steps, feature dimension)

y = y.astype(int).reshape(-1,1) # reshape y into (num_of_samples, 1)

print('Input x shape: ', x.shape)
print('Label y shape: ', y.shape)

Before we split the data into train and test sets, let's look at how many samples have seizures and how many samples do not.

In [ ]:
# Count #seizures vs #non-seizures in y
# Remember that 1 means seizure and 0 means non-seizure
### YOUR CODE HERE

### END CODE

### Discussion: Imbalanced Data

We can see that our data is imbalanced, i.e. there are much more non-seizure cases than seizure cases. Imbalanced data is common in healthcare domains --- there are usually more healthy cases than diseased cases.

Would this be a potential problem for machine learning models?

### Exercise: Split the data into training and test sets
Split the data into training and test sets. Make the training vs test set ratio = `80%/20%`. You may set `random_state=1` for reproducibility.

In [ ]:
### YOUR CODE HERE

### END CODE

print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print('x_test shape: ', x_test.shape)
print('y_test shape: ', y_test.shape)

Count the number of seizures vs non-seizures in training and test sets.

In [ ]:
### YOUR CODE HERE

### END CODE

print('Number of seizures in train set: ', num_seizure_train)
print('Number of non-seizures in train set: ', num_nonseizure_train)

print('Number of seizures in test set: ', num_seizure_test)
print('Number of non-seizures in test set: ', num_nonseizure_test)

### Let's build a vanilla RNN!

Let's build an RNN with the following architecture:

*   2 RNN layers with 32 RNN units
*   1 fully connected (i.e. dense) layer

You may find the following Keras functions useful (already imported):
*   `SimpleRNN(units = number_of_rnn_units, return_sequences = True)`
*   `SimpleRNN(units = number_of_rnn_units)`
*   `Dense(units = number_of_output_neurons, activation = 'sigmoid')`

In [ ]:
# Define an RNN with the 2 RNN layers and one fully connected layer

rnn = Sequential()
### YOUR CODE HERE

### END CODE

Compile and train the RNN.

In [ ]:
# Compile the RNN
rnn.compile(loss='binary_crossentropy',
            optimizer = 'adam', 
            metrics = ['accuracy'])

# Train the RNN
rnn.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, callbacks=[monitor])

Predict on the test data.

In [ ]:
# Predict on the test data
### YOUR CODE HERE

### END CODE

print('Test accuracy: ', accuracy_score(y_test, predictions))

### Exercise:
Try **adding more RNN layers** and/or **varying the number of RNN units** and observe the model performance.

## Long Short Term Memory (LSTM)
The performance of vanilla RNNs often degrades when the input sequence is long because
* The activation function in RNNs is non-linear --- information gets zeroed out and washed away for long sequences.
*  Vanilla RNNs are **not selective** when it comes to **what to keep** and **what to forget** --- all information is read and all information is overwritten.

Hence, we need models that can choose what to keep and what to forget! LSTMs allow us to do that.

LSTM is a special type of RNN that is designed to learn **long-term dependencies**. They were first introduced by Hochreiter & Schmidhuber in 1997 ([paper](https://www.bioinf.jku.at/publications/older/2604.pdf)).

LSTMs have **two** forms of memory --- **cell state** and **hidden state**. LSTMs control the information flow into and out of the cell through the three **gates** --- **input, forget** and **output gates**.

A schematic of the LSTM cell is shown below:
![LSTM](https://drive.google.com/uc?id=17b7oxigMmGcyx19fyzCKF-pf6dIBMLVM)

Now, let's see if we can further improve the seizure detection accuracy with a LSTM! 

Our LSTM has 2 LSTM layers with 32 units and 1 fully connected layer. The following Keras functions may be useful:
*  `LSTM(units = number_of_rnn_units)`
*  `Dense(units = number_of_output_neurons, activation = 'sigmoid')`



In [ ]:
### YOUR CODE HERE
# Build a LSTM with 2 LSTM layers and one fully connected layer

# Compile the model

# Train the model

# Predict on test data

### END CODE
print('Test accuracy: ', accuracy_score(y_test, predictions))

Woohoo! Higher test accuracy using LSTM!

## Many-to-many Sequence Modeling

So far, we have seen **many-to-one** sequence modeling using a vanilla RNN and a LSTM.

In addition to many-to-one modeling, RNNs are also capable of **many-to-many** modeling --- we can get an output from RNN at each time step.

Now, let's build a many-to-many LSTM with 2 LSTM layers and one fully connected layer with sigmoid activation.



### Exercise

Build a many-to-many LSTM with 2 LSTM layers (32 RNN units) and one fully connected layer with sigmoid activation.

**Hint:** The keras LSTM layer has an argument `return_sequences`. Setting it to be `True` will give you output at each time step.

In [ ]:
### YOUR CODE HERE
# Build a LSTM with 2 LSTM layers and one fully connected layer

# Compile the model

# Train the model

### END CODE

In [ ]:
# Get predicted probabilities on test set
proba = lstm_many2many.predict(x_test)
print('proba shape: ', proba.shape)

**Question:**  Can you explain the shape of `proba`?

In our dataset, for each of the 500 samples, in addition to one combined seizure/non-seizure label for 23 seconds (i.e. 23 time steps), we also have fine-grained labels for each time step. The labels are saved in `y_time_steps`.

In [ ]:
# Get labels for each time step in test set
y_test_time_steps = y_time_steps[test_indices]
print(y_test_time_steps.shape)

## **(Advanced)** Model Explainability --- Visualizing Model Confidence Over Time

Now that we have the model's predicted probabilities for each time step, we can visualize the model's confidence over time, and compare it to the ground truth labels over time.

### Exercise

Write a function to visualize the model's predicted probabilities and the labels over time for one sample in the same plot.

Use red color for probabilities and blue color for labels. You may find the function `plt.plot` useful.

In [ ]:
def viz_model_conf_over_time(proba, label):
  """
  Args:
    proba: list of predicted probabilities for each time step for one sample, shape (23,)
    label: list of labels for each time step for one sample, shape (23,)
  """

  ### YOUR CODE HERE

  ### END CODE

Let's randomly choose 10 examples from the test set and visualize them. Feel free to visualize more examples by modifying `range(10)`.

In [ ]:
# Randomly choose 10 samples from the test set and visualize them
for _ in range(10):
  rand_index = np.random.randint(len(y_test))
  print(rand_index)
  viz_model_conf_over_time(proba[rand_index], y_test_time_steps[rand_index])

### Discussion

From the above visualization,
* How's our model performing in general?
* What is the trend in the model's confidence over time?
* Do you see any mistake made by the model?

## Final Remarks
**Congratulations!** You have successfully completed the project!

Our models achieved >90% accuracy. Are they good enough for deployment to hospitals to assist the doctors? Most likely not. 

The real-world scenario is much more complicated. The seizure EEGs we use here are from single EEG channels (i.e. electrodes) that have been annotated as seizures. In clinical settings, EEGs are usually recorded from multiple EEG electrodes. Our models need to be able to detect seizures from multi-channel EEGs without being explicitly given the single channel data.

Moreover, the dataset we use here is from a single institution. EEGs from a different institution might be very different from what we have seen here. **Can our models generalize to data from new institutions?** In fact, making AI models clinically applicable and generalizable to unseen data is still an open question and an active field of research.

Finally, deploying medical AI models faces many other issues, one of which is **ethics**. For example, how to ensure patients' privacy? Should a doctor fully trust an AI model's decisions? If there is a mis-diagnosis by the AI model, who should take the responsibility? 

Nevertheless, do not let the challenges stop us. We are extremely excited about the future of AI in healthcare!
